In [ ]:
import h5py
import matplotlib.pyplot as plt
import hdf5plugin
import numpy as np
import scipy.ndimage

def get_mu(h5_file_path):
    mu = []
    with h5py.File(h5_file_path, 'r') as file:
        for item in file['1.1/instrument/mu']:
            item_str = str(item)
            mu.append(file['1.1/instrument/mu/' + item_str][()])
    return mu

def image_stack_from_hdf(filename):
    try:
        with h5py.File(filename, 'r') as f:
            dataset = f['entry_0000/measurement/data']
            image_stack = np.array(dataset)

            for i, frame in enumerate(image_stack):
                noise_floor = 107  # got better result with 107
                image_stack[i][image_stack[i] < noise_floor] = 0

            for i, frame in enumerate(image_stack):
                sigma = 1
                image_stack[i] = scipy.ndimage.gaussian_filter(frame, sigma)

            return image_stack

    except Exception as e:
        print("An error occurred:", e)
        return None

def com(image_stack, mu_list):
    max_indices = np.argmax(image_stack, axis=0)

    mu_array = np.array(mu_list).flatten()
    mu_map = mu_array[max_indices]

   
    mu_map_normalized = (mu_map - np.min(mu_map))

   
    midpoint = np.median(mu_map_normalized)
    mu_map_normalized -= midpoint

    
    top_rows_to_mask = 560
    bottom_rows_to_mask = 780

   
    mu_map_normalized[:top_rows_to_mask, :] = np.nan
    mu_map_normalized[-bottom_rows_to_mask:, :] = np.nan

   
    vmin = -np.max(np.abs(mu_map_normalized))
    vmax = np.max(np.abs(mu_map_normalized))

    
    height, width = mu_map_normalized.shape
    extent = [0, width * 0.15, height * 0.15, 0]

   
    fig, ax = plt.subplots(figsize=(10, 8))
    cax = ax.imshow(mu_map_normalized, cmap='RdBu', vmin=vmin, vmax=vmax, extent=extent)
    cbar = fig.colorbar(cax)

    # Set font sizes
    ax.tick_params(axis='both', which='major', labelsize=20)
    cbar.ax.tick_params(labelsize=20)
    ax.set_xlabel('X (µm)', fontsize=20)
    ax.set_ylabel('Y (µm)', fontsize=20)
    ax.set_title('Mu Map (Normalized, Centered at Zero)', fontsize=20)

    plt.tight_layout()
    plt.show()

    return max_indices, mu_map_normalized

In [ ]:

image_stack_path = r'C:\Users\Admin\Downloads\Fe_C_19_2keVrocking\scan0012\pco_ff_0000.h5'
mu_list_path = r'C:\Users\Admin\Downloads\Fe_C_19_2keVrocking\Fe-C_19_2_keV_10x_layer_rocking.h5'


image_stack = image_stack_from_hdf(image_stack_path)
mu_list = get_mu(mu_list_path)

In [ ]:

image_stack_path = 'G:/My Drive/Fe-C-H_farfield/Fe-C-H_farfield_10x_mosalayers_corrected/scan0012/pco_ff_0000.h5'
mu_list_path = 'G:/My Drive/Fe-C-H_farfield/Fe-C-H_farfield_10x_mosalayers_corrected/Fe-C-H_farfield_10x_mosalayers_corrected.h5'


image_stack = image_stack_from_hdf(image_stack_path)
mu_list = get_mu(mu_list_path)

In [ ]:

def create_composite_image(image_stack):
    """
    Sum all images in the stack to create a composite dark field image.
    This enhances contrast features present across multiple frames.
    """
    if image_stack is None:
        print("Error: No image stack provided")
        return None
    
    print(f"Summing {len(image_stack)} images...")
    
    
    composite_image = np.sum(image_stack, axis=0)
    
    
    composite_image = composite_image.astype(np.float64)
    
    
    composite_image = scipy.ndimage.gaussian_filter(composite_image, sigma=0.8)
    
    p2, p98 = np.percentile(composite_image, (2, 98))
    composite_image = np.clip(composite_image, p2, p98)
    composite_image = (composite_image - p2) / (p98 - p2)
    
    return composite_image

composite_image = create_composite_image(image_stack)

if composite_image is not None:
   
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    
    ax1.imshow(image_stack[len(image_stack)//2], cmap='gray')
    ax1.set_title('Single Frame (Middle)', fontsize=14)
    ax1.set_xlabel('Pixels', fontsize=12)
    ax1.set_ylabel('Pixels', fontsize=12)
    
    
    im2 = ax2.imshow(composite_image, cmap='gray')
    ax2.set_title('Composite Image (All Frames Summed)', fontsize=14)
    ax2.set_xlabel('Pixels', fontsize=12)
    ax2.set_ylabel('Pixels', fontsize=12)
    
    
    plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Composite image shape: {composite_image.shape}")
    print(f"Pixel size: 0.15 μm/pixel")
    print(f"Field of view: {composite_image.shape[1] * 0.15:.1f} × {composite_image.shape[0] * 0.15:.1f} μm²")
else:
    print("Failed to create composite image")

In [ ]:

def calculate_dislocation_density(image, pixel_size_um=0.15, grid_spacing_pixels=50, 
                                threshold_method='otsu', morphology_operations=True, crop_region=None):

    from skimage import filters, feature, morphology, measure
    from scipy import ndimage
    
    if image is None:
        return None, None, None
    
    
    if crop_region is not None:
        y_start, y_end = crop_region
        original_shape = image.shape
        image = image[y_start:y_end, :]
        print(f"Image cropped from {original_shape} to {image.shape}")
        print(f"Cropped region: rows {y_start}-{y_end} (vertical: {y_start*pixel_size_um:.1f}-{y_end*pixel_size_um:.1f} μm)")
    
   
    from skimage import exposure
    
   
    p2, p98 = np.percentile(image, (2, 98))
    img_stretched = exposure.rescale_intensity(image, in_range=(p2, p98))
    
    img_enhanced = exposure.equalize_adapthist(img_stretched, clip_limit=0.03)
    
    working_image = image
    threshold_method_clean = threshold_method
        
    
    # Analyze image statistics
    img_mean = np.mean(working_image)
    img_std = np.std(working_image)
    img_min = np.min(working_image)
    img_max = np.max(working_image)
    
    print(f"Image statistics - Mean: {img_mean:.4f}, Std: {img_std:.4f}")
    print(f"Image range - Min: {img_min:.4f}, Max: {img_max:.4f}")
    
    
    if threshold_method_clean == 'otsu':
        threshold = filters.threshold_otsu(working_image)
        print(f"Otsu threshold: {threshold:.4f}")
    
        # Local adaptive thresholding
        from skimage.filters import threshold_local
        threshold_map = threshold_local(working_image, block_size=51, method='mean', offset=0.01)
        edges = working_image > threshold_map
        threshold = np.mean(threshold_map)  
        print(f"Local Otsu threshold (mean): {threshold:.4f}")
    
    
    
    if threshold_method_clean != 'local_otsu':
        edges = working_image > threshold
    
    print(f"Percentage of pixels above threshold: {np.sum(edges) / edges.size * 100:.2f}%")
    
    
    edges_dis = edges.copy()  
 

    edges_for_analysis = edges_dis
    
    
    
    height, width = image.shape
    grid_lines = []
    
    # Horizontal lines
    for y in range(grid_spacing_pixels, height, grid_spacing_pixels):
        grid_lines.append(('horizontal', y, 0, width))
    
    # Vertical lines
    for x in range(grid_spacing_pixels, width, grid_spacing_pixels):
        grid_lines.append(('vertical', 0, x, height))
    
    print(f"Created {len(grid_lines)} grid lines")
    
    
    
    total_intersections = 0
    total_grid_length_pixels = 0
    
    for line_type, start_coord, fixed_coord, end_coord in grid_lines:
        if line_type == 'horizontal':
            
            y = start_coord
            if 0 <= y < height:
                line_pixels = edges_for_analysis[y, :]  
                total_grid_length_pixels += width
                
                intersections = np.sum(line_pixels)  
                total_intersections += intersections
        else:  
            x = fixed_coord
            if 0 <= x < width:
                line_pixels = edges_for_analysis[:, x]  
                
                intersections = np.sum(line_pixels)  
                total_intersections += intersections
    
    
    
    
    feature_density = np.sum(edges_for_analysis) / edges_for_analysis.size
    print(f"Feature density: {feature_density:.4f} ({feature_density*100:.2f}% of image)")
    
  
    expected_intersections_alt = int(total_grid_length_pixels * feature_density)
    print(f"Expected intersections from density: {expected_intersections_alt}")
    
    
    # Convert measurements to micrometers
    total_grid_length_um = total_grid_length_pixels * pixel_size_um
    
    
    if total_grid_length_um > 0:
        # Base calculation
        base_density_per_um = total_intersections / (2 * total_grid_length_um)
        

        
    
        fine_grid_factor = 3.0 
        
        
        darkfield_factor = 2.5  
        
    
        
        
        print(f"  Base density: {base_density_per_um:.2e} /μm")
        
        
    
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))
    
    axes[0,0].imshow(image, cmap='gray')
    axes[0,0].set_title('Original Composite Image', fontsize=14)
    axes[0,0].set_xlabel('Pixels', fontsize=12)
    axes[0,0].set_ylabel('Pixels', fontsize=12)
    
    axes[0,1].imshow(working_image, cmap='gray')
    axes[0,1].set_title('Preprocessed Image', fontsize=14)
    axes[0,1].set_xlabel('Pixels', fontsize=12)
    axes[0,1].set_ylabel('Pixels', fontsize=12)
    
    
    axes[1,0].hist(image.flatten(), bins=100, alpha=0.7, density=True, label='Original', color='blue')
    axes[1,0].hist(working_image.flatten(), bins=100, alpha=0.7, density=True, label='Preprocessed', color='orange')
    axes[1,0].axvline(threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {threshold:.4f}')
    axes[1,0].set_title('Intensity Histograms with Threshold', fontsize=14)
    axes[1,0].set_xlabel('Intensity', fontsize=12)
    axes[1,0].set_ylabel('Density', fontsize=12)
    axes[1,0].legend()
    
    
    axes[2,1].imshow(edges_for_analysis, cmap='gray', alpha=0.9)
    
   
    grid_lines_to_show = []
    
    height, width = edges_for_analysis.shape
    
    # Horizontal lines
    for y in range(grid_spacing_pixels, height, grid_spacing_pixels):
        grid_lines_to_show.append(('horizontal', y))
    
    # Vertical lines  
    for x in range(grid_spacing_pixels, width, grid_spacing_pixels):
        grid_lines_to_show.append(('vertical', x))
    
    print(f"Total grid lines to display: {len(grid_lines_to_show)}")
    

    for i, (line_type, coord) in enumerate(grid_lines_to_show[::step]):
        if line_type == 'horizontal':
            axes[2,1].axhline(y=coord, color='red', alpha=0.7, linewidth=0.8)
        else:
            axes[2,1].axvline(x=coord, color='red', alpha=0.7, linewidth=0.8)
    
    axes[2,1].set_title(f'Grid Analysis ({total_intersections} intersections, {grid_spacing_pixels}px={grid_spacing_pixels*pixel_size_um:.0f}nm spacing)', fontsize=14)
    axes[2,1].set_xlabel('Pixels', fontsize=12)
    axes[2,1].set_ylabel('Pixels', fontsize=12)
    
    plt.tight_layout()
    plt.show()